# Medical Image Classification with CNNs
This notebook demonstrates importing, preprocessing, and training CNN models on the NIH Chest X-rays dataset using transfer learning. Models like VGG16, ResNet50, and EfficientNet are utilized.

In [ ]:
# Import necessary libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, ResNet50, EfficientNetB0
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix

## Load and Preprocess the Dataset
Assuming the NIH Chest X-rays dataset is downloaded and stored locally.

In [ ]:
# Set dataset paths (adjust paths as needed)
data_dir = '/path/to/chest_xray_dataset'  # Update to your dataset path
train_dir = os.path.join(data_dir, 'train')
test_dir = os.path.join(data_dir, 'test')


In [ ]:
# Data Augmentation and Preprocessing
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)
test_datagen = ImageDataGenerator(rescale=1.0/255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'  # Adjust as needed
)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

## Define CNN Model Using Transfer Learning
We will use VGG16, ResNet50, and EfficientNetB0 architectures for transfer learning.

In [ ]:
# Define model creation function
def build_model(base_model):
    base_model.trainable = False
    x = base_model.output
    x = Flatten()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(1, activation='sigmoid')(x)
    return Model(inputs=base_model.input, outputs=predictions)

# Initialize models
vgg16_base = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
resnet50_base = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
efficientnet_base = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

models = {
    'VGG16': build_model(vgg16_base),
    'ResNet50': build_model(resnet50_base),
    'EfficientNetB0': build_model(efficientnet_base)
}

## Train and Evaluate the Models
Compile, train, and evaluate each model on the dataset.

In [ ]:
# Compile and train models
for name, model in models.items():
    print(f'Training {name}...')
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
    history = model.fit(
        train_generator,
        epochs=10,
        validation_data=test_generator,
        verbose=1
    )

    # Evaluate model
    print(f'Evaluating {name}...')
    test_loss, test_accuracy = model.evaluate(test_generator)
    print(f'{name} Test Loss: {test_loss}, Test Accuracy: {test_accuracy}')

    # Generate classification report and confusion matrix
    y_true = test_generator.classes
    y_pred = np.round(model.predict(test_generator)).astype(int)
    print(classification_report(y_true, y_pred))
    cm = confusion_matrix(y_true, y_pred)
    sns.heatmap(cm, annot=True, fmt='d')
    plt.title(f'{name} Confusion Matrix')
    plt.show()